## Importamos funciones

In [ ]:
import requests
import pandas as pd
import json

## Realizamos consulta a la API de rapidapi para obtener estadísticas

- Obtenemos estadísticas para enriquecer nuestro dataframe, en este caso, tapones y asistencia de la temporada 2019-2020 de todo los jugadores de la NBA

In [3]:
url = "https://free-nba.p.rapidapi.com/stats?seasons[]=2019"

querystring = {"page":"0","per_page":"200"}

headers = {
    'x-rapidapi-host': "free-nba.p.rapidapi.com",
    'x-rapidapi-key': "38d64eff57msh466e4e8e29e6a52p1a2fb0jsnab2f85994546"
    }

response = requests.get(url, headers=headers, params=querystring)

print(response.text)
data = response.json()

{"data":[{"id":1448930,"ast":8,"blk":1,"dreb":8,"fg3_pct":20.0,"fg3a":5,"fg3m":1,"fg_pct":36.8,"fga":19,"fgm":7,"ft_pct":75.0,"fta":4,"ftm":3,"game":{"id":62585,"date":"2019-10-22T00:00:00.000Z","home_team_id":13,"home_team_score":112,"period":4,"postseason":false,"season":2019,"status":"Final","time":"     ","visitor_team_id":14,"visitor_team_score":102},"min":"36:00","oreb":1,"pf":3,"player":{"id":237,"first_name":"LeBron","height_feet":6,"height_inches":8,"last_name":"James","position":"F","team_id":14,"weight_pounds":250},"pts":18,"reb":9,"stl":1,"team":{"id":14,"abbreviation":"LAL","city":"Los Angeles","conference":"West","division":"Pacific","full_name":"Los Angeles Lakers","name":"Lakers"},"turnover":5},{"id":1448935,"ast":1,"blk":1,"dreb":3,"fg3_pct":0.0,"fg3a":0,"fg3m":0,"fg_pct":33.3,"fga":3,"fgm":1,"ft_pct":100.0,"fta":1,"ftm":1,"game":{"id":62585,"date":"2019-10-22T00:00:00.000Z","home_team_id":13,"home_team_score":112,"period":4,"postseason":false,"season":2019,"status":"F

In [5]:
data["data"][0]


{'id': 1448930,
 'ast': 8,
 'blk': 1,
 'dreb': 8,
 'fg3_pct': 20.0,
 'fg3a': 5,
 'fg3m': 1,
 'fg_pct': 36.8,
 'fga': 19,
 'fgm': 7,
 'ft_pct': 75.0,
 'fta': 4,
 'ftm': 3,
 'game': {'id': 62585,
  'date': '2019-10-22T00:00:00.000Z',
  'home_team_id': 13,
  'home_team_score': 112,
  'period': 4,
  'postseason': False,
  'season': 2019,
  'status': 'Final',
  'time': '     ',
  'visitor_team_id': 14,
  'visitor_team_score': 102},
 'min': '36:00',
 'oreb': 1,
 'pf': 3,
 'player': {'id': 237,
  'first_name': 'LeBron',
  'height_feet': 6,
  'height_inches': 8,
  'last_name': 'James',
  'position': 'F',
  'team_id': 14,
  'weight_pounds': 250},
 'pts': 18,
 'reb': 9,
 'stl': 1,
 'team': {'id': 14,
  'abbreviation': 'LAL',
  'city': 'Los Angeles',
  'conference': 'West',
  'division': 'Pacific',
  'full_name': 'Los Angeles Lakers',
  'name': 'Lakers'},
 'turnover': 5}

- De la consulta anterior sabemos que tenemos 283 páginas, cada una con 100 registros por página por lo que iteramos y almacenamos en listas los valores

In [ ]:
blk = []
stl = []
for i in range(284):
    #print(i)
    url = "https://free-nba.p.rapidapi.com/stats?seasons[]=2019"

    querystring = {"page":"{i}","per_page":"100"}

    headers = {'x-rapidapi-host': "free-nba.p.rapidapi.com",
               'x-rapidapi-key': "38d64eff57msh466e4e8e29e6a52p1a2fb0jsnab2f85994546"}

    res = requests.get(url, headers=headers, params=querystring)
    data = response.json()
    
    for i in data["data"]:
        #players.append(f'{i["player"]["first_name"]} , {i["player"]["last_name"]}')
        blk.append(f'{i["blk"]}')
        stl.append(f'{i["stl"]}')
        

In [ ]:
players = []

for i in range(284):
    #print(i)
    url = "https://free-nba.p.rapidapi.com/stats?seasons[]=2019"

    querystring = {"page":"{i}","per_page":"100"}

    headers = {'x-rapidapi-host': "free-nba.p.rapidapi.com",
               'x-rapidapi-key': "38d64eff57msh466e4e8e29e6a52p1a2fb0jsnab2f85994546"}

    res = requests.get(url, headers=headers, params=querystring)
    data = response.json()
    
    for i in data["data"]:
        players.append(f'{i["player"]["first_name"]} , {i["player"]["last_name"]}')
        

In [10]:
print(len(players))
print(len(blk))
print(len(stl))

28400
28400
28400


- Generamos df con los resultados de la consuta para luego megearlo con nuestro df

In [135]:
df_req = pd.DataFrame.from_dict({"player": players, "blk": blk, "stl": stl})
df_req.head(10)

,player,blk,stl
0,"LeBron , James",1,1
1,"Dwight , Howard",1,0
2,"Lou , Williams",0,1
3,"Jared , Dudley",0,0
4,"JaVale , McGee",2,0
5,"Patrick , Beverley",1,0
6,"Danny , Green",1,2
7,"Patrick , Patterson",0,0
8,"Avery , Bradley",0,0
9,"Kawhi , Leonard",1,2


In [30]:
len(df_req.player.unique())

100

In [47]:
df_req["blk"] = df_req.blk.apply(lambda x: int(x))
df_req["stl"] = df_req.stl.apply(lambda x: int(x))
df_req["player"] = df_req.player.apply(lambda x: x.replace(" , ", " "))

In [56]:
df_req = df_req.groupby("player")["blk", "stl"].mean()
df_req.head()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,blk,stl
player,,
Abdel Nader,0,0
Alex Caruso,0,0
Alex Len,1,2
Andre Drummond,1,4
Anfernee Simons,0,2


- Cuando ya tenemos los datos en el formato que necesitamos, cargamos el dataframe original

## Cargamos el dataframe del proyecto

In [57]:
df = pd.read_csv("input/all_seasons.csv")
df.head(10)

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,0,Dennis Rodman,CHI,36.0,198.12,99.790240,Southeastern Oklahoma State,USA,1986,2,...,5.7,16.1,3.1,16.1,0.186,0.323,0.100,0.479,0.113,1996-97
1,1,Dwayne Schintzius,LAC,28.0,215.90,117.933920,Florida,USA,1990,1,...,2.3,1.5,0.3,12.3,0.078,0.151,0.175,0.430,0.048,1996-97
2,2,Earl Cureton,TOR,39.0,205.74,95.254320,Detroit Mercy,USA,1979,3,...,0.8,1.0,0.4,-2.1,0.105,0.102,0.103,0.376,0.148,1996-97
3,3,Ed O'Bannon,DAL,24.0,203.20,100.697424,UCLA,USA,1995,1,...,3.7,2.3,0.6,-8.7,0.060,0.149,0.167,0.399,0.077,1996-97
4,4,Ed Pinckney,MIA,34.0,205.74,108.862080,Villanova,USA,1985,1,...,2.4,2.4,0.2,-11.2,0.109,0.179,0.127,0.611,0.040,1996-97
5,5,Eddie Johnson,HOU,38.0,200.66,97.522280,Illinois,USA,1981,2,...,8.2,2.7,1.0,4.1,0.034,0.126,0.220,0.541,0.102,1996-97
6,6,Eddie Jones,LAL,25.0,198.12,86.182480,Temple,USA,1994,1,...,17.2,4.1,3.4,4.1,0.035,0.091,0.209,0.559,0.149,1996-97
7,7,Elden Campbell,LAL,28.0,213.36,113.398000,Clemson,USA,1990,1,...,14.9,8.0,1.6,3.3,0.095,0.183,0.222,0.520,0.087,1996-97
8,8,Eldridge Recasner,ATL,29.0,193.04,86.182480,Washington,USA,1992,Undrafted,...,5.7,1.6,1.3,-0.3,0.036,0.076,0.172,0.539,0.141,1996-97
9,9,Elliot Perry,MIL,28.0,182.88,72.574720,Memphis,USA,1991,2,...,6.9,1.5,3.0,-1.2,0.018,0.081,0.177,0.557,0.262,1996-97


In [58]:
df.season.unique()

array(['1996-97', '1997-98', '1998-99', '1999-00', '2000-01', '2001-02',
       '2002-03', '2003-04', '2004-05', '2005-06', '2006-07', '2007-08',
       '2008-09', '2009-10', '2010-11', '2011-12', '2012-13', '2013-14',
       '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20'],
      dtype=object)

In [59]:
df.shape

(11145, 22)

In [60]:
df.draft_round.unique()

array(['2', '1', '3', 'Undrafted', '4', '8', '6', '7'], dtype=object)

In [61]:
df[df.season == '2019-20'].shape

(514, 22)

In [62]:
df.columns

Index(['Unnamed: 0', 'player_name', 'team_abbreviation', 'age',
       'player_height', 'player_weight', 'college', 'country', 'draft_year',
       'draft_round', 'draft_number', 'gp', 'pts', 'reb', 'ast', 'net_rating',
       'oreb_pct', 'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct', 'season'],
      dtype='object')

In [93]:
df3 = df.drop(['Unnamed: 0', 'college', 'draft_year', 'draft_round', 'net_rating', 'draft_number', 'usg_pct', 'ast_pct', 'oreb_pct', 'dreb_pct'], axis=1)
df3 = df3[df3.season == '2019-20']
df3.head()

,player_name,team_abbreviation,age,player_height,player_weight,country,gp,pts,reb,ast,ts_pct,season
10631,Georges Niang,UTA,26.0,200.66,104.326160,USA,57,5.8,1.9,0.6,0.615,2019-20
10632,George Hill,MIL,33.0,190.50,85.275296,USA,51,9.7,3.0,2.9,0.675,2019-20
10633,Gary Trent Jr.,POR,21.0,195.58,94.800728,USA,52,7.8,1.7,0.9,0.563,2019-20
10634,Gary Payton II,WAS,27.0,190.50,88.450440,USA,27,4.2,2.9,1.7,0.473,2019-20
10635,Gary Harris,DEN,25.0,193.04,95.254320,USA,54,10.3,2.9,2.1,0.513,2019-20


### Cuando ya tenemos el dataframe limpio, hacemos un merge con los datos de las requests a la API 

In [127]:
df4 = pd.merge(df3, df_req, left_on='player_name', right_on='player', how="left")
df4.head()

,player_name,team_abbreviation,age,player_height,player_weight,country,gp,pts,reb,ast,ts_pct,season,blk,stl
0,Georges Niang,UTA,26.0,200.66,104.326160,USA,57,5.8,1.9,0.6,0.615,2019-20,0.0,0.0
1,George Hill,MIL,33.0,190.50,85.275296,USA,51,9.7,3.0,2.9,0.675,2019-20,NaN,NaN
2,Gary Trent Jr.,POR,21.0,195.58,94.800728,USA,52,7.8,1.7,0.9,0.563,2019-20,0.0,0.0
3,Gary Payton II,WAS,27.0,190.50,88.450440,USA,27,4.2,2.9,1.7,0.473,2019-20,NaN,NaN
4,Gary Harris,DEN,25.0,193.04,95.254320,USA,54,10.3,2.9,2.1,0.513,2019-20,0.0,2.0


In [128]:
df4.stl.value_counts()

0.0    64
1.0    23
2.0     8
4.0     2
5.0     1
3.0     1
Name: stl, dtype: int64

In [129]:
df4["stl"] = df4.stl.fillna(0.0)
df4["blk"] = df4.blk.fillna(0.0)

In [130]:
df4.isnull().sum()

player_name          0
team_abbreviation    0
age                  0
player_height        0
player_weight        0
country              0
gp                   0
pts                  0
reb                  0
ast                  0
ts_pct               0
season               0
blk                  0
stl                  0
dtype: int64

In [131]:
df4.shape

(514, 14)

In [132]:
df4.columns

Index(['player_name', 'team_abbreviation', 'age', 'player_height',
       'player_weight', 'country', 'gp', 'pts', 'reb', 'ast', 'ts_pct',
       'season', 'blk', 'stl'],
      dtype='object')

## Organizamos las columnas y las renombramos

In [133]:
df5 = df4[['player_name', 'team_abbreviation', 'age','player_height', 'player_weight', 'country', 'gp', 'pts', 'reb', 'ast', 'blk', 'stl', 'ts_pct', 'season']]
df5.columns = ['nombre', 'iniciales_equipo', 'edad', 'altura_cm', 'peso_kg', 'pais', 'part_jug', 'pts', 'reb', 'ast', 'blk', 'stl', 'pct_tiro', 'temporada']

In [134]:
df5.head()

,nombre,iniciales_equipo,edad,altura_cm,peso_kg,pais,part_jug,pts,reb,ast,blk,stl,pct_tiro,temporada
0,Georges Niang,UTA,26.0,200.66,104.326160,USA,57,5.8,1.9,0.6,0.0,0.0,0.615,2019-20
1,George Hill,MIL,33.0,190.50,85.275296,USA,51,9.7,3.0,2.9,0.0,0.0,0.675,2019-20
2,Gary Trent Jr.,POR,21.0,195.58,94.800728,USA,52,7.8,1.7,0.9,0.0,0.0,0.563,2019-20
3,Gary Payton II,WAS,27.0,190.50,88.450440,USA,27,4.2,2.9,1.7,0.0,0.0,0.473,2019-20
4,Gary Harris,DEN,25.0,193.04,95.254320,USA,54,10.3,2.9,2.1,0.0,2.0,0.513,2019-20


## Exportamos el dataframe definitivo en csv

In [136]:
df5.to_csv("output/csv_definitivo.csv", index = False)